In [1]:
# Calculating the SVD
# Algorithm from Golub and Loan, Matrix Computations, 4th Edition Johns Hopkins University Press (2013)
# 2 steps
# 1. Bidiagonalization using Householder reflectors
# 2. Reduction to diagonal form using Givens rotations
import numpy as np

In [3]:

def givens(a,b):
    if b==0:
        c = 1
        s = 0
    else:
        if abs(b) > abs(a):
            t = -a/b
            s = 1/np.sqrt(1+t**2)
            c = s*t
        else:
            t = -b/a
            c = 1/np.sqrt(1+t**2)
            s = c*t
    return c,s

In [4]:
# Sample test matrix
A = np.array([[-1.0,1.0],[2.0,-2.0],[-2.0,4.0]])
              
print(A.shape)
print(A)

(3, 2)
[[-1.  1.]
 [ 2. -2.]
 [-2.  4.]]


In [6]:
# 1. step 1. Bidiagonalization
a1 = A[:,0]
a1 = np.reshape(a1,(3,1))
print("a1 shape ",a1.shape)
print(a1)
c = np.linalg.norm(a1)
e1 = np.array([[1.0],[0.0],[0.0]])
#print(e1.shape)
v = a1 - c*e1
#print("v", v)
In = np.eye(3)
#print(In)
V = -2*np.dot(v,v.T)/(np.dot(v.T,v))
#print("V shape ", V.shape)
#print(V)
U1c = In+V
print("U1c shape ", U1c.shape)
B1 = np.dot(U1c,A)
print("B1")
print(B1.round(2))

a1 shape  (3, 1)
[[-1.]
 [ 2.]
 [-2.]]
U1c shape  (3, 3)
B1
[[ 3.   -4.33]
 [ 0.    0.67]
 [-0.    1.33]]


In [7]:

a2= B1[1:,1]
a2 = np.reshape(a2,(2,1))
#print(a2)
c2 = np.linalg.norm(a2)
e1 = np.array([[1.0],[0.0]])
v2 = a2 - c2*e1
V2 = -2*np.dot(v2,v2.T)/(np.dot(v2.T,v2))
In2 = np.eye(2)
H2c = In2 + V2
#print(H2c)
U2c = np.eye(3)
U2c[1:,1:] = H2c
print("U2c shape ",U2c.shape)
B2 = np.dot(U2c,B1)
print(B2.round(2))

U2c shape  (3, 3)
[[ 3.   -4.33]
 [-0.    1.49]
 [ 0.    0.  ]]


In [9]:
T = np.dot(B2,B2.T)
T11 = B2[0,0]**2 
T12 = B2[0,0]*B2[0,1]
T21 = T12
T22 = B2[1,1]**2 + B2[0,1]**2
Tmn = np.array([[T11, T12],[T21, T22]])
print(Tmn)

[[  9. -13.]
 [-13.  21.]]


In [10]:
# Step 2 apply Givens rotations to reduce to diagonal form

we,ve = np.linalg.eig(Tmn)
#print("eigenvalues ", we)
#print(" target ", T22)
le= we[1]
#print(" eig ", le)
a = T11 - le
b = T12
print("a, b in G1 ", a,b, " T11 ", T11, " le ", le)
cg,sg = givens(a,b)
#print(cg,sg)
G1 = np.array([[cg,sg],[-sg,cg]])
print("G1 first time")
print(G1.round(2))
B2s = B2[:2,:]   # square matrix
B3 = np.dot(B2s,G1)
print("B3 ")
print(B3.round(2))
cg,sg = givens(B3[0,0],B3[1,0])
G2 = np.array([[cg,-sg],[sg,cg]])
B4 = np.dot(G2,B3)
print("B4 ") # already hold the singular values
print(B4.round(2))

a, b in G1  -20.31782106327635 -13.0  T11  9.0  le  29.31782106327635
G1 first time
[[ 0.84 -0.54]
 [ 0.54  0.84]]
B3 
[[ 0.19 -5.27]
 [ 0.8   1.26]]
B4 
[[-0.83  0.  ]
 [-0.   -5.41]]


In [11]:

G2e = np.concatenate((G2,np.zeros((2,1))),axis=1)
G2e = G2e.T
onet = np.array([0,0,1])
onet = np.reshape(onet,(3,1))

G2e = np.concatenate((G2e,onet),axis=1)
print("G2e ")
print(G2e.round(2))
Uc = np.dot(U2c.T,G2e)
Uc = np.dot(U1c.T,Uc)

G2e 
[[-0.23  0.97  0.  ]
 [-0.97 -0.23  0.  ]
 [ 0.    0.    1.  ]]


In [12]:
# check answer against numpy's SVD
U,S,VT = np.linalg.svd(A)
print()
print("SVD of A ")
print("U")
print(U.round(2))
print("U calculated ")
print(Uc.round(2))
print(S.round(2))
print("VT")
print(VT.round(2))
print("G1")
print(G1.round(2))
print("U1c ")
print(U1c.round(2))


SVD of A 
U
[[-0.26  0.37 -0.89]
 [ 0.51 -0.73 -0.45]
 [-0.82 -0.57 -0.  ]]
U calculated 
[[ 0.37 -0.26  0.89]
 [-0.73  0.51  0.45]
 [-0.57 -0.82  0.  ]]
[5.41 0.83]
VT
[[ 0.54 -0.84]
 [-0.84 -0.54]]
G1
[[ 0.84 -0.54]
 [ 0.54  0.84]]
U1c 
[[-0.33  0.67 -0.67]
 [ 0.67  0.67  0.33]
 [-0.67  0.33  0.67]]


In [16]:
# Computations appear to agree well except for sign and order
print("numpy.linalg singular values ")
print(S.round(2))
print(" computed singular values ")
print(B4.round(2))

numpy.linalg singular values 
[5.41 0.83]
 computed singular values 
[[-0.83  0.  ]
 [-0.   -5.41]]
